# Geoparser with Address Normalization
### By **Néstor Suat** in 2021

**Descripción:** Combinando `9 Agregando location` y `9 Address Normalization`.
Se realiza normalización de direcciones usando librería en C `libpostal`

Fuente: https://github.com/openvenues/libpostal

In [1]:
from postal.expand import expand_address
from ast import literal_eval

import pandas as pd
import re
import math

In [2]:
def add_location(entities):
    loc = [ t for (t,l) in entities  if l == 'loc' ]
    loc = ' '.join(loc)
    return loc

def address_normalization(location):    
    expansions = expand_address(location, 
                                roman_numerals=False, 
                                split_alpha_from_numeric=True, 
                                expand_numex=False,
                                languages=["es"])
    expansions = re.sub(r'\b(\w+)( \1\b)+', r'\1', expansions[-1])
    return expansions

### Importando dataset

In [3]:
## Variables para importar modelos y demás
dir_ = "../../data/v1/NER/src/prueba_bad_location/"

file = 'ner_dataset_test_bad.tsv' # Dataset

In [4]:
## Importando Dataset
dataset = pd.read_csv(dir_+file, delimiter = "\t", quoting = 3)
dataset.entities = dataset.entities.apply(literal_eval)
del dataset['Unnamed: 0']
del dataset['gmap']
print(dataset.shape)
dataset.head(5)

(16, 4)


,id_tweet,text,created_at,entities
0,1058331896298520576,Un muerto y cuatro heridos en Fatal accidente ...,2018-11-02 7:16:01,"[(avenida Boyacá con 71 sur, loc)]"
1,1060077229043052545,@NoticiasCaracol accidente en la kr 10 con cl ...,2018-11-07 2:51:21,"[(kr 10 con cl 1 c, loc), (estacion de TM del ..."
2,1099689733956034561,@Citytv anoche ocurrió un accidente en frente ...,2019-02-24 10:17:18,"[(universidad nacional, loc), (puente, loc), (..."
3,1125722482059763713,Av circunvalar con calle 53 accidente de dos m...,2019-05-07 6:22:09,"[(Av circunvalar con calle 53, loc)]"
4,1131707280767033344,#MovilidadBogotá #acueducto #Trancón #Accide...,2019-05-23 18:43:36,"[(3 horas, time), (la calera, loc), (circunval..."


In [5]:
dataset['location'] = dataset.entities.apply(add_location)
dataset['location']

0                           avenida Boyacá con 71 sur
1     kr 10 con cl 1 c estacion de TM del policaparpa
2     universidad nacional puente crr 30 con calle 45
3                         Av circunvalar con calle 53
4                        la calera circunvalar con 85
5                                      cll 80 Siberia
6                       autopista norte con calle 153
7                                Calle 13 con Cra 104
8                                     Americas por 50
9                                  Carrera 20 por 140
10                                  Carrera 30 por 53
11                                  Av. Boyacá por 11
12                                    calle 63 Por 24
13                                              la 80
14                           avenida NQS con calle 19
15                   CALI X 15 FRIGORIFICO SAN MARTIN
Name: location, dtype: object

## Addresss Normalization
* Usando librería en C `libpostal`

In [6]:
dataset['address_normalization'] = dataset.location.apply(address_normalization)
dataset['address_normalization']

0                             avenida boyaca con 71 sur
1     carrera 10 con calle 1 c estacion de transmile...
2     universidad nacional puente carrera 30 con cal...
3              avenida circunvalar con avenida calle 53
4                  la calera avenida circunvalar con 85
5                              avenida calle 80 siberia
6                 autopista norte con avenida calle 153
7              avenida calle 13 con avenida carrera 104
8                        avenida de las americas por 50
9                                    carrera 20 por 140
10                            avenida carrera 30 por 53
11                                avenida boyaca por 11
12                              avenida calle 63 por 24
13                                     avenida calle 80
14                     avenida nqs con avenida calle 19
15    avenida ciudad de cali con 15 frigorifico san ...
Name: address_normalization, dtype: object

In [7]:
dataset.to_csv(dir_+"ner_dataset_test_bad_norm.tsv",sep='\t')

### Generando para Geocoding

In [7]:
locations = []
for i in range(len(dataset)):
    loc = []
    norm = 'Bogotá '+dataset.iloc[i]['address_normalization']
    loc += [norm] + ['CO']
    locations.append(loc)
locations

[['Bogotá avenida boyaca con 71 sur', 'CO'],
 ['Bogotá carrera 10 con calle 1 c estacion de transmilenio del policaparpa',
  'CO'],
 ['Bogotá universidad nacional puente carrera 30 con calle 45', 'CO'],
 ['Bogotá avenida circunvalar con avenida calle 53', 'CO'],
 ['Bogotá la calera avenida circunvalar con 85', 'CO'],
 ['Bogotá avenida calle 80 siberia', 'CO'],
 ['Bogotá autopista norte con avenida calle 153', 'CO'],
 ['Bogotá avenida calle 13 con avenida carrera 104', 'CO'],
 ['Bogotá avenida de las americas por 50', 'CO'],
 ['Bogotá carrera 20 por 140', 'CO'],
 ['Bogotá avenida carrera 30 por 53', 'CO'],
 ['Bogotá avenida boyaca por 11', 'CO'],
 ['Bogotá avenida calle 63 por 24', 'CO'],
 ['Bogotá avenida calle 80', 'CO'],
 ['Bogotá avenida nqs con avenida calle 19', 'CO'],
 ['Bogotá avenida ciudad de cali con 15 frigorifico san martin', 'CO']]

In [15]:
df = pd.DataFrame(locations,columns=['address','iso2'])
#df.rename(columns={0:'address',1:''}, inplace=True)
df

,address,iso2
0,Bogotá avenida boyaca con 71 sur,CO
1,Bogotá carrera 10 con calle 1 c estacion de tr...,CO
2,Bogotá universidad nacional puente carrera 30 ...,CO
3,Bogotá avenida circunvalar con avenida calle 53,CO
4,Bogotá la calera avenida circunvalar con 85,CO
5,Bogotá avenida calle 80 siberia,CO
6,Bogotá autopista norte con avenida calle 153,CO
7,Bogotá avenida calle 13 con avenida carrera 104,CO
8,Bogotá avenida de las americas por 50,CO
9,Bogotá carrera 20 por 140,CO


In [17]:
df.to_csv(dir_+"ner_dataset_test_bad_norm_format.csv", index=False)

## Formateando resultados de Batch Geocode con Addresss Normalization

In [8]:
batch_geocode = pd.read_csv(dir_+'geocode_results_2021_03_08.csv')
batch_geocode = batch_geocode[['best_lat','best_long']]
batch_geocode

,best_lat,best_long
0,4.546440,-74.137025
1,NaN,NaN
2,4.638194,-74.084046
3,4.675313,-74.109147
4,4.617212,-74.061498
5,4.735036,-74.131319
6,4.742830,-74.048192
7,4.671054,-74.151988
8,4.625684,-74.108363
9,NaN,NaN


In [9]:
dataset['gmap'] = ''
for i in range(len(dataset)):
    if(not math.isnan(batch_geocode.iloc[i]['best_lat'])):
        dataset.at[i,'gmap'] = "{'lat': "+str(batch_geocode.iloc[i]['best_lat'])+", 'lng': "+str(batch_geocode.iloc[i]['best_long'])+"}"
    else:
        dataset.drop(index=dataset.index[[i]])

In [10]:
geocoding = dataset.copy()
geocoding

,id_tweet,text,created_at,entities,location,address_normalization,gmap
0,1058331896298520576,Un muerto y cuatro heridos en Fatal accidente ...,2018-11-02 7:16:01,"[(avenida Boyacá con 71 sur, loc)]",avenida Boyacá con 71 sur,avenida boyaca con 71 sur,"{'lat': 4.546440400000002, 'lng': -74.13702470..."
1,1060077229043052545,@NoticiasCaracol accidente en la kr 10 con cl ...,2018-11-07 2:51:21,"[(kr 10 con cl 1 c, loc), (estacion de TM del ...",kr 10 con cl 1 c estacion de TM del policaparpa,carrera 10 con calle 1 c estacion de transmile...,
2,1099689733956034561,@Citytv anoche ocurrió un accidente en frente ...,2019-02-24 10:17:18,"[(universidad nacional, loc), (puente, loc), (...",universidad nacional puente crr 30 con calle 45,universidad nacional puente carrera 30 con cal...,"{'lat': 4.6381938, 'lng': -74.08404639999998}"
3,1125722482059763713,Av circunvalar con calle 53 accidente de dos m...,2019-05-07 6:22:09,"[(Av circunvalar con calle 53, loc)]",Av circunvalar con calle 53,avenida circunvalar con avenida calle 53,"{'lat': 4.675313, 'lng': -74.109147}"
4,1131707280767033344,#MovilidadBogotá #acueducto #Trancón #Accide...,2019-05-23 18:43:36,"[(3 horas, time), (la calera, loc), (circunval...",la calera circunvalar con 85,la calera avenida circunvalar con 85,"{'lat': 4.6172124000000005, 'lng': -74.06149835}"
5,1147555712521908225,"@InviasOficial ,@numeral767 ,@BogotaseMueve . ...",2019-07-06 12:19:37,"[(cll 80, loc), (Siberia, loc)]",cll 80 Siberia,avenida calle 80 siberia,"{'lat': 4.73503645, 'lng': -74.13131915}"
6,1149638960328478721,#AEstaHora Colapsada autopista norte con calle...,2019-07-12 6:17:42,"[(autopista norte con calle 153, loc)]",autopista norte con calle 153,autopista norte con avenida calle 153,"{'lat': 4.7428300000000005, 'lng': -74.0481921}"
7,1060683217383182338,@SectorMovilidad choque simple de tractomula c...,2018-11-08 18:59:20,"[(Calle 13 con Cra 104, loc)]",Calle 13 con Cra 104,avenida calle 13 con avenida carrera 104,"{'lat': 4.6710544999999986, 'lng': -74.1519878}"
8,761823976700121088,"Americas por 50, volcamiento de bus del sitp, ...",2016-08-06 2:19:24,"[(Americas por 50, loc)]",Americas por 50,avenida de las americas por 50,"{'lat': 4.625684099999999, 'lng': -74.1083629}"
9,770763722210045956,Carrera 20 por 140 volcamiento de un vehículo ...,2016-08-30 18:22:46,"[(Carrera 20 por 140, loc)]",Carrera 20 por 140,carrera 20 por 140,


In [13]:
geocoding = geocoding[dataset['gmap']!='']
geocoding = geocoding.reset_index(drop=True)
geocoding

,id_tweet,text,created_at,entities,location,address_normalization,gmap
0,1058331896298520576,Un muerto y cuatro heridos en Fatal accidente ...,2018-11-02 7:16:01,"[(avenida Boyacá con 71 sur, loc)]",avenida Boyacá con 71 sur,avenida boyaca con 71 sur,"{'lat': 4.546440400000002, 'lng': -74.13702470..."
1,1099689733956034561,@Citytv anoche ocurrió un accidente en frente ...,2019-02-24 10:17:18,"[(universidad nacional, loc), (puente, loc), (...",universidad nacional puente crr 30 con calle 45,universidad nacional puente carrera 30 con cal...,"{'lat': 4.6381938, 'lng': -74.08404639999998}"
2,1125722482059763713,Av circunvalar con calle 53 accidente de dos m...,2019-05-07 6:22:09,"[(Av circunvalar con calle 53, loc)]",Av circunvalar con calle 53,avenida circunvalar con avenida calle 53,"{'lat': 4.675313, 'lng': -74.109147}"
3,1131707280767033344,#MovilidadBogotá #acueducto #Trancón #Accide...,2019-05-23 18:43:36,"[(3 horas, time), (la calera, loc), (circunval...",la calera circunvalar con 85,la calera avenida circunvalar con 85,"{'lat': 4.6172124000000005, 'lng': -74.06149835}"
4,1147555712521908225,"@InviasOficial ,@numeral767 ,@BogotaseMueve . ...",2019-07-06 12:19:37,"[(cll 80, loc), (Siberia, loc)]",cll 80 Siberia,avenida calle 80 siberia,"{'lat': 4.73503645, 'lng': -74.13131915}"
5,1149638960328478721,#AEstaHora Colapsada autopista norte con calle...,2019-07-12 6:17:42,"[(autopista norte con calle 153, loc)]",autopista norte con calle 153,autopista norte con avenida calle 153,"{'lat': 4.7428300000000005, 'lng': -74.0481921}"
6,1060683217383182338,@SectorMovilidad choque simple de tractomula c...,2018-11-08 18:59:20,"[(Calle 13 con Cra 104, loc)]",Calle 13 con Cra 104,avenida calle 13 con avenida carrera 104,"{'lat': 4.6710544999999986, 'lng': -74.1519878}"
7,761823976700121088,"Americas por 50, volcamiento de bus del sitp, ...",2016-08-06 2:19:24,"[(Americas por 50, loc)]",Americas por 50,avenida de las americas por 50,"{'lat': 4.625684099999999, 'lng': -74.1083629}"
8,873824438684311552,Carrera 30 por 53 choque entre dos vehículos d...,2017-06-11 3:49:15,"[(Carrera 30 por 53, loc)]",Carrera 30 por 53,avenida carrera 30 por 53,"{'lat': 4.608336299999999, 'lng': -74.0970212}"
9,1099717090376728577,En la Av. Boyacá por 11 se presentó una colisi...,2019-02-24 12:06:00,"[(Av. Boyacá por 11, loc)]",Av. Boyacá por 11,avenida boyaca por 11,"{'lat': 4.6513764, 'lng': -74.1262885}"


In [15]:
geocoding.to_csv(dir_+"ner_dataset_test_bad_norm_geocoding.tsv",sep='\t')